In [1]:
import numpy as np
import numpy.typing as npt
from typing import Any
from dataclasses import dataclass

CoordList = tuple[tuple[float,float], ...]

# Define Input Interface

@dataclass
class Material:
    board_k: float
    post_k: float
    bridge_k: float

    board_m: float
    post_m: float
    bridge_m: float

    board_alp: float
    post_alp: float
    bridge_alp: float

    board_bet: float
    post_bet: float
    bridge_bet: float

@dataclass
class Geometry:
    board_profile: npt.NDArray[np.int8]
    wall_profile: npt.NDArray[np.int8]
    holes_profile: npt.NDArray[np.int8]

    bridge_location: npt.NDArray[np.int8]
    post_location: npt.NDArray[np.int8]
    
    wall_height: float
    scale_factor: float

@dataclass
class StringTuning:
    string_tension: float
    string_mass_per_length: float

@dataclass
class Input:
    note_location: float
    impulse_location: float
    impulse_amplitude: float

# Define Instrument

class Acoustic:


    class Vibration:
        M_matrix: npt.NDArray[np.float64]
        K_matrix: npt.NDArray[np.float64]

        alpha: float
        beta: float


    class BackBoard(Vibration):
        wall_elements_pixel: npt.NDArray[np.int8]
        wall_elements_tuple: CoordList

        interior_elements_pixel: npt.NDArray[np.int8]
        interior_elements_tuple: CoordList

        k: float
        m: float
        diag_spring_ratio: float

        def __init__(self, m: float, k: float, alpha: float, beta: float,
                     diag_spring_ratio: float,
                     wall_pixel_data: npt.NDArray[np.int8],
                     interior_pixel_data: npt.NDArray[np.int8]):

            self.wall_elements_pixel = wall_pixel_data
            self.interior_elements_pixel = interior_pixel_data

            self.m = m
            self.k = k
            self.diag_spring_ratio = diag_spring_ratio

            self.parse_pixel_array()
            self.assemble_M()
            self.assemble_K()

            return

        def parse_pixel_array(self) -> None:

            self.wall_elements_tuple = np.argwhere(self.wall_elements_pixel == 0).tolist()
            self.interior_elements_tuple = np.argwhere(self.interior_elements_pixel == 0).tolist()

            return
        
        def assemble_M(self) -> None:

            dimension: int = len(self.interior_elements_tuple)
            self.M_matrix = np.diag(np.full(dimension, self.m))

            return
        
        def assemble_K(self) -> None:

            dimension: int = len(self.interior_elements_tuple)
            global_K: npt.NDArray[np.float64] = np.zeros((dimension, dimension))

            for index, (x_coord, y_coord) in enumerate(self.interior_elements_tuple):

                adj_list: CoordList = (
                    (x_coord, y_coord + 1),
                    (x_coord, y_coord - 1),
                    (x_coord + 1, y_coord),
                    (x_coord - 1, y_coord)
                )

                diag_adj_list: CoordList = (
                    (x_coord + 1, y_coord + 1),
                    (x_coord - 1, y_coord + 1),
                    (x_coord + 1, y_coord - 1),
                    (x_coord - 1, y_coord - 1)
                )

                for adj_coord in adj_list:

                    if adj_coord in self.interior_elements_tuple:

                        adj_index = self.interior_elements_tuple.index(adj_coord)
                        global_K[index,index] += self.k
                        global_K[index,adj_index] -= self.k/2
                        global_K[adj_index,index] -= self.k/2
                    
                    if adj_coord in self.wall_elements_tuple:

                        global_K[index,index] += self.k
                
                for diag_adj_coord in diag_adj_list:

                    if diag_adj_coord in self.interior_elements_tuple:

                        diag_adj_index = self.interior_elements_tuple.index(diag_adj_coord)
                        global_K[index,index] += self.k*self.diag_spring_ratio
                        global_K[index,diag_adj_index] -= self.k/2 * self.diag_spring_ratio
                        global_K[diag_adj_index,index] -= self.k/2 * self.diag_spring_ratio
                    
                    if diag_adj_coord in self.wall_elements_tuple:

                        global_K[index,index] += self.k * self.diag_spring_ratio
            
            self.K_matrix = global_K
            
            return


    class FrontBoard(Vibration):

        wall_elements_pixel: npt.NDArray[np.int8]
        wall_elements_tuple: CoordList

        interior_elements_pixel: npt.NDArray[np.int8]
        interior_elements_tuple: CoordList

        hole_elements_pixel: npt.NDArray[np.int8]
        hole_elements_tuple: CoordList

        k: float
        m: float
        diag_spring_ratio: float

        def __init__(self, m: float, k: float, alpha: float, beta: float,
                     diag_spring_ratio: float,
                     wall_pixel_data: npt.NDArray[np.int8],
                     interior_pixel_data: npt.NDArray[np.int8],
                     hole_pixel_data: npt.NDArray[np.int8]):

            self.wall_elements_pixel = wall_pixel_data
            self.interior_elements_pixel = interior_pixel_data
            self.hole_elements_pixel = hole_pixel_data

            self.m = m
            self.k = k
            self.diag_spring_ratio = diag_spring_ratio

            self.parse_pixel_array()
            self.assemble_M()
            self.assemble_K()

            return

        def parse_pixel_array(self) -> None:

            self.wall_elements_tuple = np.argwhere(self.wall_elements_pixel == 0).tolist()
            self.interior_elements_tuple = np.argwhere((self.interior_elements_pixel == 0) and (self.hole_elements_pixel != 0)).tolist()
            self.hole_elements_tuple = np.argwhere(self.hole_elements_pixel == 0).tolist()

            return
        
        def assemble_M(self) -> None:

            dimension: int = len(self.interior_elements_tuple)
            self.M_matrix = np.diag(np.full(dimension, self.m))

            return
        
        def assemble_K(self) -> None:

            dimension: int = len(self.interior_elements_tuple)
            global_K: npt.NDArray[np.float64] = np.zeros((dimension, dimension))

            for index, (x_coord, y_coord) in enumerate(self.interior_elements_tuple):

                adj_list: CoordList = (
                    (x_coord, y_coord + 1),
                    (x_coord, y_coord - 1),
                    (x_coord + 1, y_coord),
                    (x_coord - 1, y_coord)
                )

                diag_adj_list: CoordList = (
                    (x_coord + 1, y_coord + 1),
                    (x_coord - 1, y_coord + 1),
                    (x_coord + 1, y_coord - 1),
                    (x_coord - 1, y_coord - 1)
                )

                for adj_coord in adj_list:

                    if adj_coord in self.interior_elements_tuple:

                        adj_index = self.interior_elements_tuple.index(adj_coord)
                        global_K[index,index] += self.k
                        global_K[index,adj_index] -= self.k/2
                        global_K[adj_index,index] -= self.k/2
                    
                    if adj_coord in self.wall_elements_tuple:

                        global_K[index,index] += self.k
                
                for diag_adj_coord in diag_adj_list:

                    if diag_adj_coord in self.interior_elements_tuple:

                        diag_adj_index = self.interior_elements_tuple.index(diag_adj_coord)
                        global_K[index,index] += self.k*self.diag_spring_ratio
                        global_K[index,diag_adj_index] -= self.k/2 * self.diag_spring_ratio
                        global_K[diag_adj_index,index] -= self.k/2 * self.diag_spring_ratio
                    
                    if diag_adj_coord in self.wall_elements_tuple:

                        global_K[index,index] += self.k * self.diag_spring_ratio
            
            self.K_matrix = global_K
            
            return


    class Post(Vibration):
        def __init__(self, post_location:CoordList, post_k:float):
            #TODO
            return

    class Bridge(Vibration):
        def __init__(self, bridge_location:CoordList, bridge_k:float):
            #TODO
            return

    class String(Vibration):
        def __init__(self, string_tension:float, string_density:float):
            #TODO
            return

    front_board: FrontBoard
    back_board: Board
    bridge: Bridge
    post: Post
    string: String

    def __init__(self, design: Geometry, material:Material, tuning:String):
        #TODO
        return

    def export_sound(self, export_path:str, input: Input):
        #TODO
        return

    def assemble_instrument(self):
        #TODO
        front_board_M = self.front_board.M_matrix
        back_board_M = self.back_board.M_matrix
        bridge_M = self.bridge.M_matrix
        post_M = self.post.M_matrix
        string_M = self.string.M_matrix

        front_board_K = self.front_board.K_matrix
        back_board_K = self.back_board.K_matrix
        bridge_K = self.bridge.K_matrix
        post_K = self.post.K_matrix
        string_K = self.string.K_matrix
        return
    
    def system_response(self):
        #TODO
        return
    
    def calculate_sound_paths(self):
        #TODO
        return
    
    def calculate_time_delay(self):
        #TODO
        return
    
    def superimpose_sound_sources(self):
        #TODO
        return



    

SyntaxError: invalid syntax (Temp/ipykernel_23180/716137495.py, line 66)

In [ ]:
import